# Load Data

In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import accuracy_score
import itertools

In [9]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR, SVC
from sklearn import linear_model

In [10]:
idx = pd.read_csv('data/res_10.csv')


In [11]:
idx['ID'].unique()

array(['LUACTRUU Index', 'SPX Index', 'BCOM Index', 'LBUSTRUU Index',
       'LF98TRUU Index', 'LUATTRUU Index', 'LUMSTRUU Index',
       'LBEATREU Index', 'LP01TREU Index', 'LG30TRUU Index',
       'EMUSTRUU Index', 'LGTRTRUU Index', 'LGDRTRUU Index',
       'LUGCTRUU Index', 'LP06TREU Index', 'LF94TRUU Index',
       'LACHTRUU Index', 'LD08TRUU Index', 'LC07TRUU Index',
       'USYC2Y10 Index', 'LECPTREU Index'], dtype=object)

In [12]:
from data.load import get_y

y_train, y_test = get_y('SPX Index', 'LUACTRUU Index')


# Fit models

In [13]:
results_data = []

In [14]:
from dc_df.model_registry import HMMExogenousDC
from data.save_to_mlflow import save_to_mlflow

# exogenous model Random Forest with HMM

regressor_rf = RandomForestRegressor(n_jobs=12)
hmm_exogenous = HMMExogenousDC(estimator=regressor_rf, y_train=y_train, y_test=y_test, name= "RF_HMM" )
y_pred, X = hmm_exogenous.fit_predict()
accuracy, f1,fpr, tpr, area_under_the_curve = hmm_exogenous.evaluate()
results_data.append(
    ("RF_HMM", accuracy, f1,fpr[1], tpr[1], area_under_the_curve)
)

MLFLOW_RUN_ID = 1
save_to_mlflow(model_name=hmm_exogenous.get_name(), dts_name='SPX-LUACTRUU', accuracy=accuracy, f1=f1, fpr=fpr, tpr=tpr, auc=area_under_the_curve)


Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'
Even though the 'covars_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'c'
Even though the 'startprob_' attribute is set, it will be overwritten during initialization because 'init_params' contains 's'
Even though the 'transmat_' attribute is set, it will be overwritten during initialization because 'init_params' contains 't'
Even though the 'means_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'm'
Even though the 'covars_' attribute is set, it will be overwritten during initialization because 'init_params' contains 'c'


TypeError: expected string or bytes-like object

In [ ]:
# Random Forest Regression
from dc_df.base import RegressorDF
regressor_rf = RandomForestRegressor(n_jobs=12)

regressor_dc_rf  = RegressorDF(estimator=regressor_rf, y_train=y_train, y_test=y_test)

rf_pred = regressor_dc_rf.fit_predict()

accuracy, f1,fpr, tpr, area_under_the_curve = regressor_dc_rf.evaluate()
results_data.append(
    ("RF", accuracy, f1,fpr[1], tpr[1], area_under_the_curve)
)


In [ ]:
# SVR
regressor_svr = SVR()
regressor_df_svr  = RegressorDF(estimator=regressor_svr, y_train=y_train, y_test=y_test)

svr_pred = regressor_df_svr.fit_predict()

accuracy, f1,fpr, tpr, area_under_the_curve = regressor_df_svr.evaluate()
results_data.append(
    ("SVR", accuracy, f1,fpr[1], tpr[1], area_under_the_curve)
)

In [ ]:
# SVR HMM
regressor_svr = SVR()
svr_hmm = HMMExogenousDC(estimator=regressor_svr, y_train=y_train, y_test=y_test )
y_pred, X = svr_hmm.fit_predict()

accuracy, f1,fpr, tpr, area_under_the_curve = svr_hmm.evaluate()
results_data.append(
    ("SVR_HMM", accuracy, f1,fpr[1], tpr[1], area_under_the_curve)
)

In [ ]:
#Lasso regression
regressor_lasso = linear_model.Lasso()
regressor_dc_lasso  = RegressorDF(estimator=regressor_lasso, y_train=y_train, y_test=y_test)
lasso_pred  = regressor_dc_lasso.fit_predict()

accuracy, f1,fpr, tpr, area_under_the_curve = regressor_dc_lasso.evaluate()
results_data.append(
    ("LASSO", accuracy, f1,fpr[1], tpr[1], area_under_the_curve)
)

# Summary Statistics

In [ ]:
summary = pd.DataFrame(columns=['estimator','accuracy','f1','fpr','tpr','auc'], data=results_data)
summary